## Usage

This is a helper notebook. Run it from another notebook as:

```
%run ../common/benchmark_analysis.ipynb 
```

## Input

Set `data_absolute_path` to the file with JSON data to be analyzed.
This data should be generated using the `notsofine::benchmark_run` from `harness`.

In [57]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

plt.style.use('seaborn-whitegrid')

In [58]:
# Uncomment for debugging.

# import os
# data_absolute_path = os.path.join(os.getcwd(), '..', 'benchmark_trivial', 'data.json')

In [ ]:
print("### Loading data from " + data_absolute_path)

with open(data_absolute_path) as f:
    data = json.loads(f.read())
df = pd.json_normalize(data['iterations'], 'runs', ['i'])

In [ ]:
df['duration.total_ms'] = df['duration.secs'] * \
    1e3 + df['duration.nanos'] / 1e6

series = df.pivot(
    index='i',
    columns='program',
    values=['duration.total_ms'])

print('### Statistics: Raw data')
print(series.describe())
series.plot(y='duration.total_ms', kind='line',
            title='Measured time in all benchmark runs (milliseconds)')


# 

In [ ]:
# Remove outliers by dropping rows where _any_ column value is more than 2 SDs away from the column mean.

mask = (np.abs(stats.zscore(series)) < 2).all(axis=1)
series_sans_outliers = series[mask]

print('### Statistics: After outlier removal')
print(series_sans_outliers.describe())
series_sans_outliers.plot(y='duration.total_ms', kind='line',
                          title='Measured time in all benchmark runs after outlier removal (milliseconds)')


In [ ]:
baseline = series_sans_outliers.columns[0][1]
diffs = series_sans_outliers.subtract(series_sans_outliers.iloc[:, 0], axis=0).rename({
    'duration.total_ms': 'difference_ms'}, axis=1)
diffs = diffs.iloc[:, 1:]
print('## Absolute difference in runtimes compared to ' + baseline)
print(diffs.describe())
diffs.plot(y='difference_ms', kind='line',
           title='Runtime difference w.r.t.' + baseline + ' (milliseconds)')


In [ ]:
relative_diffs = diffs.div(series_sans_outliers.iloc[:, 0], axis=0).rename({
    'difference_ms': 'relative_difference'}, axis=1)
print('## Relative difference in runtimes compared to ' + baseline)
print(relative_diffs.describe())
relative_diffs.plot(y='relative_difference', kind='line',
                    title='Runtime difference w.r.t.' + baseline + ' (milliseconds)')
